In [1]:
import re
import time
import requests
import math as m
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
print('SORT ACCORDING TO - 1\n')

print('alpha')
print('user_rating')
print('num_votes')
print('boxoffice_gross_us')
print('runtime')
print('year')

SORT ACCORDING TO - 1

alpha
user_rating
num_votes
boxoffice_gross_us
runtime
year


In [3]:
print('SORT ACCORDING TO - 2\n')

print('asc')
print('desc')

SORT ACCORDING TO - 2

asc
desc


In [4]:
sort1 = input('SORT ACCORDING TO - 1 --> ')
sort2 = input('SORT ACCORDING TO - 2 --> ')
pages = int(input('Enter number of pages you want to scrap --> '))

SORT ACCORDING TO - 1 --> user_rating
SORT ACCORDING TO - 2 --> desc
Enter number of pages you want to scrap --> 8


In [5]:
movie_name = []
release_year = []
certificate = []
total_runtime = []
genre = []
imdb_rating = []
metascore = []
votes = []
gross = []

In [6]:
def get_movie_name(data):
    movies_scrap = data.find_all('h3', class_='lister-item-header')
    return movies_scrap[0].a.get_text()

In [7]:
def get_release_year(data):
    year_scrap = data.find_all('span', class_='lister-item-year text-muted unbold')
    
    year1 = year_scrap[0].get_text()
    year = re.search(r"\d{4}", year1) 
    
    return year.group()

In [8]:
def get_certificate(data):
    certificate_scrap = data.find_all('span', class_='certificate')
    
    if certificate_scrap:
        return certificate_scrap[0].get_text()
    else :
        return str(m.nan)

In [9]:
def get_total_runtime(data):
    runtime_scrap = data.find_all('span', class_='runtime')
    
    runtime1 = runtime_scrap[0].get_text()
    runtime = re.search(r"\d{3}|\d{2}", runtime1)
    
    return runtime.group()

In [10]:
def get_genre(data):
    genre_scrap = data.find_all('span', class_='genre')

    return genre_scrap[0].get_text().replace('\n', '').replace(' ','')

In [11]:
def get_imdb_rating(data):
    rating_scrap = data.find_all('div', class_='inline-block ratings-imdb-rating')

    return rating_scrap[0].strong.get_text()

In [12]:
def get_metascore(data):
    metascore_scrap = data.find_all('div', class_='inline-block ratings-metascore')
    
    if metascore_scrap:
        return metascore_scrap[0].span.get_text().replace(' ', '')
    else :
        return str(m.nan)

In [13]:
def get_vote_gross(data):
    vote_gross = data.find_all('span', attrs = {'name':'nv'})
    
    vote = vote_gross[0].get_text()
    gross = vote_gross[1].get_text().replace('$', '').replace('M', '') if len(vote_gross)>1 else str(m.nan)
    
    return vote, gross

In [14]:
def get_data_df(soup):
    data_store = soup.find_all('div', 'lister-item mode-advanced')   
    
    for i in range(len(data_store)):
        movie_name.append(get_movie_name(data_store[i]))
        
        release_year.append(get_release_year(data_store[i]))
        
        certificate.append(get_certificate(data_store[i]))
        
        total_runtime.append(get_total_runtime(data_store[i]))
        
        genre.append(get_genre(data_store[i]))
        
        imdb_rating.append(get_imdb_rating(data_store[i]))
        
        metascore.append(get_metascore(data_store[i]))
        
        votes.append(get_vote_gross(data_store[i])[0])
        
        gross.append(get_vote_gross(data_store[i])[1])

In [15]:
def get_imdb_data(sort1, sort2, pages):
    
    for i in range(pages):
        start_page = str((50*i) + 1)
        
        base_url1 = 'https://www.imdb.com/search/title/?groups=top_1000&sort='
        base_url2 = f"start={start_page}&ref_=adv_nxt"
        link = f"{base_url1}{sort1},{sort2}{base_url2}"
    
        page = requests.get(link)
        soup = bs(page.content, 'html.parser')
    
        get_data_df(soup)
    
        time.sleep(6)
       
    df = pd.DataFrame()
    
    df['Movie Name'] = movie_name 
    df['Release Year'] = release_year
    df['Certificate'] = certificate 
    df['Total Runtime(mins)'] = total_runtime
    df['Genre'] = genre
    df['IMDB Rating'] = imdb_rating
    df['Metascore'] = metascore
    df['Votes'] = votes
    df['Gross Collection($ and M)'] = gross
    
    return df

In [16]:
df = get_imdb_data(sort1, sort2, pages)
df

,Movie Name,Release Year,Certificate,Total Runtime(mins),Genre,IMDB Rating,Metascore,Votes,Gross Collection($ and M)
0,The Shawshank Redemption,1994,A,142,Drama,9.3,80,"2,429,071",28.34
1,The Godfather,1972,A,175,"Crime,Drama",9.2,100,"1,681,524",134.97
2,Soorarai Pottru,2020,U,153,Drama,9.1,nan,"85,794",nan
3,The Dark Knight,2008,UA,152,"Action,Crime,Drama",9.0,84,"2,384,930",534.86
4,The Godfather: Part II,1974,A,202,"Crime,Drama",9.0,90,"1,167,164",57.30
...,...,...,...,...,...,...,...,...,...
395,Terminator 2: Judgment Day,1991,A,137,"Action,Sci-Fi",8.5,75,"1,022,876",204.84
396,Nuovo Cinema Paradiso,1988,U,155,"Drama,Romance",8.5,80,"241,371",11.99
397,Hotaru no haka,1988,U,89,"Animation,Drama,War",8.5,94,"247,496",nan
398,Back to the Future,1985,U,116,"Adventure,Comedy,Sci-Fi",8.5,87,"1,097,434",210.61


In [17]:
df.to_csv('imdb_data.csv', index=False)